In [49]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


In [50]:
data = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [74]:


x = tf.placeholder(tf.float32)
y_true = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)



def simple_2layer_nn(x):
    W_1 = tf.Variable(tf.random_normal([784, N_NEURONS_1]))
    x_1 = tf.add(tf.matmul(x, W_1), tf.Variable(tf.random_normal([N_NEURONS_1])))
    x_1 = tf.nn.relu(x_1)

    W_2 = tf.Variable(tf.random_normal([N_NEURONS_1, N_NEURONS_2]))
    x_2 = tf.add(tf.matmul(x_1, W_2), tf.Variable(tf.random_normal([N_NEURONS_2])))
    x_2 = tf.nn.relu(x_2)


    W_3 = tf.Variable(tf.random_normal([N_NEURONS_2, 10]))
    y_pred = tf.add(tf.matmul(x_2, W_3), tf.Variable(tf.random_normal([10])))
    y_pred = tf.nn.relu(y_pred)
    
    return y_pred

def simple_2layer_nn_with_dropout(x, keep_prob):
    W_1 = tf.Variable(tf.random_normal([784, N_NEURONS_1]))
    x_1 = tf.add(tf.matmul(x, W_1), tf.Variable(tf.random_normal([N_NEURONS_1])))
    x_1 = tf.nn.relu(x_1)
    x_1 = tf.nn.dropout(x_1, keep_prob)
    
    W_2 = tf.Variable(tf.random_normal([N_NEURONS_1, N_NEURONS_2]))
    x_2 = tf.add(tf.matmul(x_1, W_2), tf.Variable(tf.random_normal([N_NEURONS_2])))
    x_2 = tf.nn.relu(x_2)
    x_2 = tf.nn.dropout(x_2, keep_prob)

    W_3 = tf.Variable(tf.random_normal([N_NEURONS_2, 10]))
    y_pred = tf.add(tf.matmul(x_2, W_3), tf.Variable(tf.random_normal([10])))
    y_pred = tf.nn.relu(y_pred)
    
    return y_pred

def simple_2layer_nn_with_dropout_xavier(x, keep_prob):
    W_1 = tf.get_variable(
        'W_1', shape=(784, N_NEURONS_1), initializer=tf.contrib.layers.xavier_initializer())
    x_1 = tf.add(tf.matmul(x, W_1), tf.Variable(tf.random_normal([N_NEURONS_1])))
    x_1 = tf.nn.relu(x_1)
    x_1 = tf.nn.dropout(x_1, keep_prob)
    
    W_2 = tf.get_variable(
        'W_2', shape=(N_NEURONS_1, N_NEURONS_2), initializer=tf.contrib.layers.xavier_initializer())
    x_2 = tf.add(tf.matmul(x_1, W_2), tf.Variable(tf.random_normal([N_NEURONS_2])))
    x_2 = tf.nn.relu(x_2)
    x_2 = tf.nn.dropout(x_2, keep_prob)

    W_3 = tf.get_variable(
        'W_3', shape=(N_NEURONS_2, 10), initializer=tf.contrib.layers.xavier_initializer())
    y_pred = tf.add(tf.matmul(x_2, W_3), tf.Variable(tf.random_normal([10])))
    y_pred = tf.nn.relu(y_pred)
    
    return y_pred

def simple_2layer_nn_sigmoid(x, keep_prob):
    W_1 = tf.Variable(tf.random_normal([784, N_NEURONS_1]))
    x_1 = tf.add(tf.matmul(x, W_1), tf.Variable(tf.random_normal([N_NEURONS_1])))
    x_1 = tf.nn.sigmoid(x_1)

    W_2 = tf.Variable(tf.random_normal([N_NEURONS_1, N_NEURONS_2]))
    x_2 = tf.add(tf.matmul(x_1, W_2), tf.Variable(tf.random_normal([N_NEURONS_2])))
    x_2 = tf.nn.sigmoid(x_2)


    W_3 = tf.Variable(tf.random_normal([N_NEURONS_2, 10]))
    y_pred = tf.add(tf.matmul(x_2, W_3), tf.Variable(tf.random_normal([10])))
    y_pred = tf.nn.sigmoid(y_pred)
    
    return y_pred

In [75]:
def train_and_compare(y_pred, x, y_true, keep_prob):
    
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

    gd_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
    correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))
    
    with tf.Session() as sess:
    
        # Train
        init = tf.global_variables_initializer()
        sess.run(init)

        for epoch in range(NUM_EPOCHS):
            avg_cost = 0.
            total_batch = int(data.train.num_examples/MINIBATCH_SIZE)

            for _ in range(total_batch):
                batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
                sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys, keep_prob: 0.5})

            if epoch % DISPLAY_STEP == 0:
                ans = sess.run(
                            accuracy, 
                            feed_dict={x: data.test.images, y_true: data.test.labels, keep_prob: 1})
                print("Epoch: {0}, accuracy: {1}".format(epoch, ans))

In [76]:
N_NEURONS_1 = 200
N_NEURONS_2 = 200
NUM_EPOCHS = 40
MINIBATCH_SIZE = 100
LEARNING_RATE = 0.0001
DISPLAY_STEP = 5
# pred = simple_2layer_nn(x)
# train_and_compare(pred, x, y_true, keep_prob)

# pred = simple_2layer_nn_with_dropout(x, keep_prob)
# train_and_compare(pred, x, y_true, keep_prob)

# pred = simple_2layer_nn_with_dropout_xavier(x, keep_prob)
# train_and_compare(pred, x, y_true, keep_prob)

pred = simple_2layer_nn_sigmoid(x, keep_prob)
train_and_compare(pred, x, y_true, keep_prob)


Epoch: 0, accuracy: 0.28600001335144043
Epoch: 5, accuracy: 0.6669999957084656
Epoch: 10, accuracy: 0.7874000072479248
Epoch: 15, accuracy: 0.8105000257492065
Epoch: 20, accuracy: 0.8216000199317932
Epoch: 25, accuracy: 0.8309000134468079
Epoch: 30, accuracy: 0.8364999890327454
Epoch: 35, accuracy: 0.8393999934196472


1) Dropout makes it longer to train the network

2) Xavier initialization makes great initial boost for the training (compared with the random initialization): in the first epoch the accuracy is already ~0.57!
